In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pykrx import stock
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, BatchNormalization, Reshape, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from datetime import datetime, timedelta

In [116]:
df = stock.get_market_ohlcv_by_date("19990106", "20210719", "000660")
volume_df = stock.get_market_trading_volume_by_date("19990106", "20210719", "000660")
del df['시가']
del df['고가']
del df['저가']
del volume_df['기타법인']
del volume_df['전체']
df = df.join(volume_df)
df = df.dropna()
print(df)

                종가      거래량       기관합계         개인      외국인합계
날짜                                                          
1999-01-06  354601  1048060    28270.0   841250.0  -869520.0
1999-01-07  407517  3325700 -1242610.0  2322640.0 -1080030.0
1999-01-08  369720  1105840   -66980.0   336250.0  -269270.0
1999-01-11  364909  1321090   -53250.0   419310.0  -366060.0
1999-01-12  364222   777710   -56680.0   205950.0  -149270.0
...            ...      ...        ...        ...        ...
2021-07-13  123000  2879072   116372.0 -1093033.0  1068500.0
2021-07-14  123500  2443087  -106894.0  -393989.0   522022.0
2021-07-15  123500  1500981    96255.0  -100524.0     4363.0
2021-07-16  121500  2905546  -417976.0   703293.0  -288411.0
2021-07-19  119000  2066638  -449113.0   877759.0  -427145.0

[5555 rows x 5 columns]


In [117]:
df_close = pd.read_csv("Data_pred.csv")
df_close.head()
print(len(df_close))
list_close = df_close["0"]
print(list_close[0])

1345
0.17234007


In [79]:
df.to_csv('Sample_3.csv', encoding = "utf-8-sig")

In [118]:
test_array = np.zeros(shape = (5, 5555))
for i in range(5):
    tmp = []
    for j in range(len(df.index)):
        tmp.append(df.iloc[j][i])
    max_tmp = max(tmp)
    min_tmp = min(tmp)
    print(max_tmp, min_tmp)
    for j in range(len(tmp)):
        tmp[j] = (tmp[j] - min_tmp)/(max_tmp - min_tmp)
    arr_tmp = np.array(tmp)
    test_array[i] = arr_tmp
    #print(norm_arr.shape)

718996.0 2650.0
1832794266.0 494550.0
9929240.0 -208355414.0
207103064.0 -27931553.0
30005613.0 -38321590.0


In [107]:
print(test_array.shape)
print(test_array[0])

(5, 5555)
[0.49131425 0.56518358 0.51241998 ... 0.16870339 0.16591145 0.16242151]


In [119]:
for i in range(1345):
    print(test_array[0][4210 + i])
    test_array[0][4210 + i] = list_close[i]
    print(test_array[0][4210 + i])

0.034899336354219886
0.17234007
0.03441074564526081
0.1656732
0.035946316444846485
0.17290951
0.036016115117554925
0.17390028
0.03462014166338613
0.17072421
0.03552752440859585
0.17451549
0.03552752440859585
0.17371054
0.03420134962713549
0.17177245
0.03448054431796925
0.17762323
0.034969135026928326
0.18305375
0.03664430317193088
0.18628497
0.038249672644225
0.18800162
0.03845906866235032
0.18824364
0.03762148458984904
0.18817768
0.03804027662609968
0.18676585
0.038738263353184074
0.18422678
0.03943625008026847
0.17999405
0.03929665273485159
0.18108548
0.03929665273485159
0.1812684
0.03817987397151656
0.19708529
0.03950604875297691
0.19868211
0.040902022207145707
0.19990332
0.040832223534437266
0.19889891
0.040832223534437266
0.19752313
0.038877860698600955
0.18994166
0.038947659371309395
0.19129476
0.03859866600776719
0.18786174
0.03915705538943472
0.18423198
0.038668464680475634
0.18171349
0.03804027662609968
0.17985189
0.03706309520818152
0.17736721
0.03713289388088996
0.18146138
0

In [120]:
fin_array = test_array.T
fin_array = fin_array.reshape(-1, 5, 5, 1)
print(fin_array.shape)

(1111, 5, 5, 1)


In [121]:
encoder_input = tf.keras.Input(shape=(5, 5, 1))

x = Conv2D(25, 1, padding='same')(encoder_input)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(50, 1, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(125, 5, strides=5, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(125, 5, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Flatten()(x)

encoder_output = Dense(2)(x)

encoder = Model(encoder_input, encoder_output)
encoder.summary()

Model: "model_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, 5, 5, 1)]         0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 5, 5, 25)          50        
_________________________________________________________________
batch_normalization_108 (Bat (None, 5, 5, 25)          100       
_________________________________________________________________
leaky_re_lu_108 (LeakyReLU)  (None, 5, 5, 25)          0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 5, 5, 50)          1300      
_________________________________________________________________
batch_normalization_109 (Bat (None, 5, 5, 50)          200       
_________________________________________________________________
leaky_re_lu_109 (LeakyReLU)  (None, 5, 5, 50)          0  

In [122]:
decoder_input = tf.keras.Input(shape=(2, ))

x = Dense(1*1*125)(decoder_input)
x = Reshape((1, 1, 125))(x)

x = Conv2DTranspose(50, 5, strides=5, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(25, 5, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(1, 1, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(1, 1, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

decoder_output = Conv2DTranspose(1, 5, strides=1, padding='same', activation='tanh')(x)

decoder = Model(decoder_input, decoder_output)
decoder.summary()

Model: "model_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_28 (Dense)             (None, 125)               375       
_________________________________________________________________
reshape_16 (Reshape)         (None, 1, 1, 125)         0         
_________________________________________________________________
conv2d_transpose_80 (Conv2DT (None, 5, 5, 50)          156300    
_________________________________________________________________
batch_normalization_112 (Bat (None, 5, 5, 50)          200       
_________________________________________________________________
leaky_re_lu_112 (LeakyReLU)  (None, 5, 5, 50)          0         
_________________________________________________________________
conv2d_transpose_81 (Conv2DT (None, 5, 5, 25)          312

In [123]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32

encoder_in = tf.keras.Input(shape=(5, 5, 1))
x = encoder(encoder_in)
decoder_out = decoder(x)

auto_encoder = Model(encoder_in, decoder_out)
auto_encoder.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE), loss=tf.keras.losses.MeanSquaredError())

In [124]:
checkpoint_path = 'tmp/result_4.ckpt'
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, save_weights_only=True, monitor='loss', verbose=1)

auto_encoder.fit(fin_array, fin_array, batch_size=BATCH_SIZE, epochs=30, callbacks=[checkpoint], )
auto_encoder.load_weights(checkpoint_path)

Epoch 1/30
35/35 [==============================] - 3s 20ms/step - loss: 0.2834

Epoch 00001: loss improved from inf to 0.28340, saving model to tmp\result_4.ckpt
Epoch 2/30
35/35 [==============================] - 1s 20ms/step - loss: 0.1989

Epoch 00002: loss improved from 0.28340 to 0.19892, saving model to tmp\result_4.ckpt
Epoch 3/30
35/35 [==============================] - 1s 23ms/step - loss: 0.1512

Epoch 00003: loss improved from 0.19892 to 0.15125, saving model to tmp\result_4.ckpt
Epoch 4/30
35/35 [==============================] - 1s 21ms/step - loss: 0.1247

Epoch 00004: loss improved from 0.15125 to 0.12470, saving model to tmp\result_4.ckpt
Epoch 5/30
35/35 [==============================] - 1s 21ms/step - loss: 0.1034

Epoch 00005: loss improved from 0.12470 to 0.10340, saving model to tmp\result_4.ckpt
Epoch 6/30
35/35 [==============================] - 1s 22ms/step - loss: 0.0849

Epoch 00006: loss improved from 0.10340 to 0.08486, saving model to tmp\result_4.ckpt
Ep

In [125]:
decoded_arr = auto_encoder.predict(fin_array)

In [126]:
a_array = fin_array.reshape(5555, 5)
b_array = decoded_arr.reshape(5555, 5)

In [127]:
# f = open("flag_1.txt", 'w')
num = np.zeros(shape = (5555, 5))
flag_1 = 0
flag_2 = 0
flag_3 = 0
flag_4 = 0
flag_5 = 0
for i in range(5555):
    for j in range(5):
        num[i][j] = abs(a_array[i][j] - b_array[i][j])
        if (num[i][j] < 0.1):
            flag_1 += 1
            # f.write("{} {}\n".format(i, j))
        elif (num[i][j] < 0.3):
            flag_2 += 1
        elif (num[i][j] < 0.5):
            flag_3 += 1
        elif (num[i][j] < 0.7):
            flag_4 += 1
        else:
            flag_5 += 1
#f.close()
print(flag_1, flag_2, flag_3, flag_4, flag_5)

27481 267 22 3 2


In [102]:
np.savetxt('AE_abs_4.csv', num, delimiter=',')